**Objective:**

A prefix-tree is an ordered tree structure which can represent a transaction database in a highly compressed form. It is constructed by reading transactions one at a time with a predefined item order and mapping each transaction onto a path in the prefix-tree. Since different transactions can have several items in common, their paths may overlap.

Assume a transaction DB of sequences. Write an efficient python implementation using a prefix tree to extract all sequence patterns that have a higher frequency than 'k'

## Approach

There's some amount of literature present on implementing prefix tree data structures efficiently for transaction dbs. My approach is: <br>
- review what literature is available to get a high level overview
- implement a prefix tree and finalise which tree architecture might make sense to you for grabbing itemset supports
- implement a vanilla support counter - don't worry about optimising it here
- iterate with different algorithms and strategies from different papers about performance gains from different data structure implementations

Since I've done market basket analysis in the past, I understand the basic concepts and only need to understand the data structures. So timebox myself to 3 hours.

## Literature Review and understanding

- https://www-users.cs.umn.edu/~kumar001/dmbook/ch6.pdf - excellent book for overall architecture
- http://www.borgelt.net/papers/sam.pdf : split and merge algo by Chris Borgelt
- https://linux.thai.net/~thep/datrie/datrie.html#WhatTake : two array implementation of a trie for better memory
- https://ac.els-cdn.com/0895717703900586/1-s2.0-0895717703900586-main.pdf?_tid=24efd131-ea62-414d-bb03-3d4a2b6ffd90&acdnat=1526709387_78cce21acd95ae58bbdaca8cbf76538b : three array implementation to be used along with apriori
- http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.87.2654&rep=rep1&type=pdf - excellent resource for trie implementation. use this
- http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.87.2654&rep=rep1&type=pdf: same author trie based implementation. the guy uses far too many arrays though. how is this memory efficient?
- https://wimleers.com/article/fp-growth-powered-association-rule-mining-with-support-for-constraints : fp growth. newer algo but faster. not much though. to be explored. i may not have time for this.
- "Efficient Implementations of Apriori and Eclat" - solid paper when i skimmed through it, explaining the data structure at an implementation level. the guy also talks about another algo Eclat, which seems to have similar gains as fp growth.


**Outcome: ** 
The vanilla implementation is obviously set up for failure because of combinatorial explosion when dealing with transaction db level of rows. We need some form of pruning for this data structure to be reasonably useful. So I will use the vanilla implementation as the baseline algorithm. 

Then, I'll start with implementing apriori algorithm for candidate generation and pruning. This is one of the oldest and most studied algorithms. I'm confident there's several production level tested implementations available. 

Ferenc Bodon's paper explains how Apriori  has reasonable traverse as compared to eclat memory requirements and outperforms fp growth when there are too many frequent items (which is probably the case for a medical knowledge base). 

## Apriori algorithm implementation using APYORI

To get a better sense of how the data structure might look, let me play around with a packaged implementation of apriori algorithm.

In [124]:
# !pip install apyori

  Running setup.py bdist_wheel for apyori ... done
  Stored in directory: /Users/vivekbharadwaj/Library/Caches/pip/wheels/7b/2a/35/c0c3749c1a36d4f454ea22d8396e1b854b86340d63cbbb7949
Successfully built apyori
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from apyori import apriori

Assume the transaction database to be a list of lists. Each sub-list is a transaction row.

In [3]:
tx_list = [['a','d'],
        ['a','b','d'],
        ['c','d'],
        ['a','b','c','d'],
        ['b','c'],
        ['d'],
        ['b','c','d']]

Calculate rows with minimum support, in this case we call it support of 2.

In [4]:
[x[0] for x in list(apriori(tx_list,min_support=2/len(tx_list)))]

[frozenset({'a'}),
 frozenset({'b'}),
 frozenset({'c'}),
 frozenset({'d'}),
 frozenset({'a', 'b'}),
 frozenset({'a', 'd'}),
 frozenset({'b', 'c'}),
 frozenset({'b', 'd'}),
 frozenset({'c', 'd'}),
 frozenset({'a', 'b', 'd'}),
 frozenset({'b', 'c', 'd'})]

Fair enough. Use sets and possibly dict of lists. Some of the papers describe using pointers.

## Imports for coding

In [5]:
import numpy as np
from itertools import combinations
from collections import defaultdict, OrderedDict

## Pointer based implementation of TRIE

In [6]:
class TrieNode(object):
    '''
    Each node object is a parent with potential children and a support count 
    '''
    
    def __init__(self):
        self.children = {}
        self.transaction_complete = False
        self.support_count = 0

In [7]:
class TxTrie(object):
    '''The entire transaction database which is a list of lists is represented 
        in a tree structure with a root node defined at depth 0, which points to 
        child nodes at depth d. Each node is represented by a label and pointers
        to children
       :param lst_Items_in_transaction: This is used in insert_items
               represents each row of the transaction database which is ordered by frequency
       :param candidate_items: This is used in search_items. In the apriori algo, a
                pruned subset of candidate item sets generated from each row of the 
                transaction database is presented to the trie for traversal and search
    '''    
    
    # initialising as root
    def __init__(self):
        self.root = TrieNode()
        
    def insert_items(self,lst_Items_in_transaction):
        # initially refers to root instance. Later it will iterate over each item in the row
        current_txn = self.root
        
        for item in lst_Items_in_transaction:
            if item[1] not in current_txn.children:
                current_txn.children[item[1]] = TrieNode()
            # current transaction goes down the rabbit hole
            current_txn = current_txn.children[item[1]]
        current_txn.transaction_complete = True
        
    def search_items(self,candidate_items):
        current_txn = self.root
        
        for item in candidate_items:
            if item not in current_txn.children:
                return 0
            current_txn = current_txn.children[item]
        if current_txn.transaction_complete == True:
            current_txn.support_count += 1
            return 1
        else:
            return 0

In [10]:
def get_frequencies(transactions, min_support=1):
    '''this function is used to get a dict of relative frequencies across the entire database
        used to generate the ordered list of trnasactions for our first traversal k=1
    '''
    frequencies = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            frequencies[item] += 1
    # we want to retain only frequent itemsets
    return {k:v for k,v in frequencies.items() if v >= min_support}

In [11]:
def _sort_transactions_by_freq(
        transactions, key_func=max, reverse_int=False,min_support=2,
        reverse_ext=False, sort_ext=True):
    '''ordering transactions by frequency, currently used for first traversal and number of 
        unique items but will be used for ordering every iteration in the future algos
    '''
    
    key_seqs = [{key_func(i) for i in sequence} for sequence in transactions]
    frequencies = get_frequencies(key_seqs)

    asorted_seqs = []
    for key_seq in key_seqs:
        if not key_seq:
            continue
        # Sort each transaction (infrequent key first)
        l = [(frequencies[i], i) for i in key_seq]
        l.sort(reverse=reverse_int)
        asorted_seqs.append(tuple(l))
    # Sort all transactions. Those with infrequent key first, first
    if sort_ext:
        asorted_seqs.sort(reverse=reverse_ext)

    return (asorted_seqs, frequencies)

In [12]:
tx_list = [['a', 'd'],
 ['a', 'b', 'd'],
 ['c', 'd'],
 ['a', 'b', 'c', 'd'],
 ['b', 'c'],
 ['d'],
 ['b', 'c', 'd']]

In [13]:
tx_listSorted, blah = _sort_transactions_by_freq(tx_list,max)
t = TxTrie()
for txn in tx_listSorted:
    t.insert_items(txn)

In [14]:
tx_listSorted

[((3, 'a'), (4, 'b'), (4, 'c'), (6, 'd')),
 ((3, 'a'), (4, 'b'), (6, 'd')),
 ((3, 'a'), (6, 'd')),
 ((4, 'b'), (4, 'c')),
 ((4, 'b'), (4, 'c'), (6, 'd')),
 ((4, 'c'), (6, 'd')),
 ((6, 'd'),)]

In [15]:
t.root.children['b'].children['c'].transaction_complete

True

In [16]:
t.search_items(('b', 'c'))

1

## Implement baseline vanilla version from scratch

In [17]:
def candidate_itemset_generator(sorted_txn_list_row,num_of_items_itemset):
    
    lst_itemset = set()
    
    # same as before, if entire db is passed in the tx, then we assume a list of list
    # otherwise we process a simple list
    
    temp = set(combinations([item[1] for item in sorted_txn_list_row], 
                            num_of_items_itemset))
    for itemset_elements in temp:
        lst_itemset.add(itemset_elements)
    return lst_itemset

In [18]:
candidate_itemset_generator(tx_listSorted[0],num_of_items_itemset=2)

{('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')}

In [19]:
tx_listSorted, F = _sort_transactions_by_freq(tx_list, min_support=2)
list(F.keys())

['d', 'a', 'b', 'c']

In [20]:
def run_apriori_vanilla(tx_list,min_support=2):
    '''
    this function creates a trie from the list of transactions.
    then it traverses the transaction table multiple times (as a baseline to apriori)
    and for each combination of itemsets (iterated by the number of items), I create 
    candidates in order to check support.
    
    I check support by running this against each transaction row and update support count
    At the end, i prune candidates which are below my minimum support count
    '''    
    k=1; F=defaultdict(int);
    tx_listSorted, F = _sort_transactions_by_freq(tx_list, min_support=min_support)
    unique_items = list(F.keys())

    # create prefix tree
    t = TxTrie()
    for txn in tx_listSorted:
        t.insert_items(txn)
        
    while k<=len(unique_items):

        for tx in tx_listSorted:    
            tx_candidate_itemset = candidate_itemset_generator(tx, num_of_items_itemset=k)
            for candidate in tx_candidate_itemset:
                F.setdefault(candidate,0)
                F[candidate] += t.search_items(candidate)
                
        # pruning candidates where support count is greater than min support value
        F = {k:v for k,v in F.items() if v >= min_support}   
        
        k += 1
        
    return F

In [21]:
blah = run_apriori_vanilla(tx_list,min_support=2)
blah

{('a', 'b', 'd'): 2,
 ('a', 'd'): 3,
 ('b', 'c'): 3,
 ('b', 'c', 'd'): 2,
 ('c', 'd'): 3,
 ('d',): 6,
 'a': 3,
 'b': 4,
 'c': 4,
 'd': 6}

## Next iteration

This will obviously result in a combinatorial explosion. 

Next iteration will reduce the number of candidates being generated using different strategies. I will use the Fk−1×F1 Method where every frequent k-itemset is composed of a frequent (k−1)-itemset and a frequent 1-itemset. Hence the overall complexity of this step is:

O(∑k|Fk−1||F1|)

In [213]:
def run_apriori(tx_list,min_support=2):
    
    # create all frequent 1-itemset and store them in an F-array
    k=1; F=defaultdict(int);
    tx_listSorted, F[k] = _sort_transactions_by_freq(tx_list, min_support=min_support)
    
    # create trie
    t = TxTrie()
    for txn in tx_listSorted:
        t.insert_items(txn)
        
    while len(F[k])>0:

        # candidate itemset store. 
        k+=1
        
        *** need to come up with a way to union them properly!!
        F[k] = F[k-1] | F[k] 
        ***
        
        for tx in tx_listSorted:    
            tx_candidate_itemset = candidate_itemset_generator(F[k], num_of_items_itemset=k)
            for candidate in tx_candidate_itemset:
                F[k][candidate] += t.search_items(candidate)
                
        # pruning candidates where support count is greater than min support value
        F[k] = {k:v for k,v in support_count.items() if v >= min_support}   
        
        return F

Out of time. :(

I've already spent more than 4 hours on this problem. Need to move on.

## Improvement strategies

- too much superfluous traversing happens each time we scan the database. We can make sure of the ordering structure within a trie to order by frequencies and then store/ access based on it
- use tabular representation using offset index as described in Chris Borgelt "Efficient Implementation of apriori and eclat"
- employ early stopping if label is found
- use intersection based pruning as described in Ference Bodon's "Trie based Apriori implementation for mining frequent item sequences"
- At each node, store the length of the longest directed path that starts from it. When searching for k-itemset candidates, we can move downward only if the maximal path length at this node is k-d. This is explained in the "Trie: an alternarive data structure for mining" paper.
- experiment with fp growth and SAM algorithms which you couldn't get to. SAM seems to have a relatively simpler implementation but you need to understand how to make the algorithm work with a trie offset array impleemntation.